## Fine-Tuning PLBart for Python code generation from English algorithmic statements

In [1]:
# Install required packages
!pip install pandas
!pip install transformers datasets sentencepiece
!pip install huggingface_hub

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00


In [2]:
# Imports
import json
import pandas as pd
from datasets import Dataset
from transformers import PLBartTokenizer, PLBartForConditionalGeneration, Seq2SeqTrainer, Seq2SeqTrainingArguments

In [3]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Load the JSON file
import pandas as pd
import json

dataset = '/content/drive/MyDrive/itrl/standardized_dataset.json'
with open(dataset, 'r') as f:
    data = json.load(f)

# Convert to pandas DataFrame
df = pd.DataFrame(data)

# Display the first few records to check
df.head()

Mounted at /content/drive


,text,code
0,Assign number two to variable A.,A = 2
1,Assign number two to variable A.,A = 2
2,Store number two in variable A,A = 2
3,Declare the variable A and store the number tw...,A = 2
4,Assign number five to variable B.,B = 5


### Preprocessing the Dataset

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import PLBartTokenizer, PLBartForConditionalGeneration

# Split data into train and valid sets
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(valid_df)

# Load the PLBart tokenizer and model
tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-python-en_XX", src_lang="en_XX", tgt_lang="python")

# Add the new token for newline and indentation to the tokenizer
tokenizer.add_tokens('__newline_indent__')

# Function to combine newline and indentation into one token
def preprocess_code(example):
    # Combine newline and indentation
    example['code'] = example['code'].replace('\n    ', '__newline_indent__')
    return example

# Apply preprocessing to standardize indentation and tokenize the data
train_dataset = train_dataset.map(preprocess_code)
valid_dataset = valid_dataset.map(preprocess_code)

# Update the model to handle the new token
model = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-python-en_XX")
model.resize_token_embeddings(len(tokenizer))

# Define a function to tokenize the data (with the assumption that newline+indent has been replaced by <newline_indent>)
def tokenize_data(example):
    inputs = tokenizer(example['text'], max_length=256, truncation=True, padding='max_length')
    outputs = tokenizer(example['code'], max_length=256, truncation=True, padding='max_length')
    inputs['labels'] = outputs['input_ids']  # Set the labels for the training task
    return inputs

# Tokenize the training dataset
tokenized_train_dataset = train_dataset.map(tokenize_data, batched=True)

# Tokenize the validation dataset
tokenized_valid_dataset = valid_dataset.map(tokenize_data, batched=True)

# Combine into a DatasetDict
dataset = DatasetDict({
    'train': tokenized_train_dataset,
    'valid': tokenized_valid_dataset
})

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/998 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

### Fine-tuning the plbart-python-en_XX model

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    report_to="none",
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=10,
)

# Create a Seq2SeqTrainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_valid_dataset,
)

trainer.train()
trainer.evaluate()

model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")
model.save_pretrained("/content/drive/MyDrive/itrl/plbart_algo2code")
tokenizer.save_pretrained("/content/drive/MyDrive/itrl/plbart_algo2code")

Epoch,Training Loss,Validation Loss
1,2.433600,1.582546
2,0.036700,0.031222
3,0.015200,0.019035
4,0.012000,0.015455
5,0.010600,0.013009
6,0.006500,0.012083
7,0.009500,0.010921
8,0.007000,0.010347
9,0.006000,0.010175
10,0.004600,0.010017


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-defau

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}


('/content/drive/MyDrive/itrl/model-ft-e10-cl256sd/tokenizer_config.json',
 '/content/drive/MyDrive/itrl/model-ft-e10-cl256sd/special_tokens_map.json',
 '/content/drive/MyDrive/itrl/model-ft-e10-cl256sd/sentencepiece.bpe.model',
 '/content/drive/MyDrive/itrl/model-ft-e10-cl256sd/added_tokens.json')

### Model Inference

In [4]:
from transformers import PLBartForConditionalGeneration, PLBartTokenizer
import torch

class ModelInference:
    def __init__(self, model_path):
        # Load the fine-tuned model and tokenizer
        self.tokenizer = PLBartTokenizer.from_pretrained(model_path)
        self.model = PLBartForConditionalGeneration.from_pretrained(model_path)
        self.model.eval()  # Set model to evaluation mode

    def generate_code(self, input_text):
        # Tokenize the input
        inputs = self.tokenizer(input_text, return_tensors="pt")

        # Generate the output
        with torch.no_grad():  # Disable gradient calculation
            output = self.model.generate(**inputs, decoder_start_token_id=self.tokenizer.lang_code_to_id["__python__"])

        # Decode the output to get the generated Python code
        generated_code = self.tokenizer.decode(output[0], skip_special_tokens=True)

        # Postprocess the generated output to replace the custom token with newline and indent
        generated_code = generated_code.replace('__newline_indent__', '\n    ')

        return generated_code

# Usage
model_path = "/content/drive/MyDrive/itrl/plbart_algo2code"
code_generator = ModelInference(model_path)

# Test the inference function with an example input
input_text = "Print\"have a good day\""
generated_code = code_generator.generate_code(input_text)

print("Generated Python Code:\n", generated_code)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Generated Python Code:
 print(" have a good day")
